(https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Cassandra1.jpeg/800px-Cassandra1.jpeg)
## Notebook to run batch Kaissandra methods

With this notebook you can run train, test or alternated train and test methods


In [1]:
import sys
import os

this_path = os.getcwd()
path = '\\'.join(this_path.split('\\')[:-1])+'\\'
if path not in sys.path:
    sys.path.insert(0, path)
    print(path+" added to python path")
else:
    print(path+" already in python path")

C:\Users\mgutierrez\root\Projects\SDC\py\kaissandra\ added to python path


In [2]:
# Import modules
import time

from kaissandra.config import configuration, retrieve_config
from kaissandra.testRNN import test_RNN
from kaissandra.trainRNN import train_RNN
from kaissandra.inputs import Data, run_train_test, build_and_test, build_and_train

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Get config structure

In [3]:
params = {'config_name':'CTESTTSFVAR',
          
          'movingWindow':100,
          'nEventsPerStat':1000,
          'lB':1300,
          'feature_keys_manual':[],
          'feature_keys_tsfresh':[i for i in range(37,48)]+[49,50]+[i for i in range(52,68)],
           'var_feat_keys':[i for i in range(68,93)],
          
          'if_build_IO':True,
          'from_stats_file':False,
          'IDweights':'0TESTTSFVAR',
          'inverse_load':True,
          
          'IDresults':'1TESTTSFVAR',
          'startFrom':-1,
          'endAt':-1,
          'save_journal':False}

config = configuration(params)

WARNING! Arguments not taken into consideration
Config file ../../config/CTESTTSFVAR.config exists. Loaded from disk


## Build inputs/outputs and train Network

In [6]:
build_and_train(config)

1 AUDCAD
stats["means_tsf_in"]
[[-1.44543904e-04 -7.09978538e-05 -2.25201835e-04 -4.39475094e-04
   2.67297699e-01 -4.39475094e-04 -2.89576556e-07 -3.96879305e-11
  -3.96866151e-11 -3.96833749e-11 -4.39475094e-07 -2.89576556e-07
  -4.02185644e-07 -6.75835446e-07 -2.88236220e-07 -2.19684925e-07
  -6.79875518e-07 -6.63260115e-07 -5.02151241e-07 -4.74046162e-07
  -3.78795458e-07 -6.02024416e-07 -4.45242073e-07 -4.89870724e-07
  -6.36632667e-07 -6.84602077e-07 -4.27112346e-08 -2.22857583e-07
  -6.33503959e-08 -1.50558554e-07 -4.89932188e-07 -8.53484473e-07
  -8.91259384e-07 -3.12087122e-07 -1.45410813e-07 -4.06943601e-07
  -8.96012971e-08 -5.59200602e-07 -7.07825610e-07 -1.22865794e-07
  -6.13531168e-07 -2.89454161e-07 -6.49192437e-07 -6.52377255e-07
  -3.01442187e-07 -2.23311341e-07 -8.10210088e-07 -3.73192506e-07
  -2.07794543e-07 -9.69660831e-08 -5.86012111e-07 -7.61587217e-07
  -2.96058449e-07 -5.18660046e-07 -4.47563255e-07 -1.35708802e-07
  -1.06248694e-08 -2.16706014e-07 -3.22450268

 11%|████████▌                                                                    | 3656/32768 [00:50<06:30, 74.51it/s]

KeyboardInterrupt: 

 11%|████████▌                                                                    | 3656/32768 [01:10<06:30, 74.51it/s]

## Build inputs/outputs and test Network

In [ ]:
build_and_test(config)

[autoreload of kaissandra.inputs failed: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\IPython\extensions\autoreload.py", line 384, in superreload
    update_generic(old_obj, new_obj)
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\IPython\extensions\autoreload.py", line 323, in update_generic
    update(a, b)
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\IPython\extensions\autoreload.py", line 278, in update_class
    if old_obj == new_obj:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
]


1 AUDCAD
stats["means_tsf_in"]
[[-1.44543904e-04 -7.09978538e-05 -2.25201835e-04 -4.39475094e-04
   2.67297699e-01 -4.39475094e-04 -2.89576556e-07 -3.96879305e-11
  -3.96866151e-11 -3.96833749e-11 -4.39475094e-07 -2.89576556e-07
  -4.02185644e-07 -6.75835446e-07 -2.88236220e-07 -2.19684925e-07
  -6.79875518e-07 -6.63260115e-07 -5.02151241e-07 -4.74046162e-07
  -3.78795458e-07 -6.02024416e-07 -4.45242073e-07 -4.89870724e-07
  -6.36632667e-07 -6.84602077e-07 -4.27112346e-08 -2.22857583e-07
  -6.33503959e-08 -1.50558554e-07 -4.89932188e-07 -8.53484473e-07
  -8.91259384e-07 -3.12087122e-07 -1.45410813e-07 -4.06943601e-07
  -8.96012971e-08 -5.59200602e-07 -7.07825610e-07 -1.22865794e-07
  -6.13531168e-07 -2.89454161e-07 -6.49192437e-07 -6.52377255e-07
  -3.01442187e-07 -2.23311341e-07 -8.10210088e-07 -3.73192506e-07
  -2.07794543e-07 -9.69660831e-08 -5.86012111e-07 -7.61587217e-07
  -2.96058449e-07 -5.18660046e-07 -4.47563255e-07 -1.35708802e-07
  -1.06248694e-08 -2.16706014e-07 -3.22450268

## Automately train and test in alternative manner

In [15]:
# Automate train/test from variation features
if_train = True
if_test = True
its = 20
configs = [config['config_name']]

configs_list = []
# load configuration files
for config_name in configs:
    configs_list.append(retrieve_config(config_name))
# run train/test
for config in configs_list:
    # set automation-specific config fields
    config['num_epochs'] = 1
    config['startFrom'] = -1
    config['endAt'] = -1
    run_train_test(config, its, if_train, if_test)

Config file ../../config/C0400.config loaded from disk
Iteration 0 of 19
IDweights: 000400
IDresults: 100400
1 AUDCAD
	s 0 of 21. From 2016.01.04 00:00:08 to 2016.10.13 15:41:59
	s 1 of 21. From 2016.10.13 17:52:00 to 2016.12.23 23:59:59
	s 2 of 21. From 2016.12.26 06:00:00 to 2016.12.30 23:58:30
	s 3 of 21. From 2017.01.02 09:00:00 to 2017.01.26 23:59:59
	s 4 of 21. From 2017.01.30 00:05:00 to 2017.09.05 23:59:59
	s 5 of 21. From 2017.09.06 20:25:01 to 2017.11.10 09:38:15
	s 6 of 21. From 2017.11.10 10:05:52 to 2017.11.24 23:58:59
	s 7 of 21. From 2017.11.27 00:05:00 to 2017.12.08 23:58:59
	s 8 of 21. From 2017.12.11 00:05:00 to 2017.12.12 04:13:07
	s 9 of 21. From 2017.12.12 09:55:53 to 2017.12.22 23:58:59
	s 10 of 21. From 2017.12.26 03:00:00 to 2017.12.29 23:58:42
	s 11 of 21. From 2018.01.02 03:00:00 to 2018.01.10 21:25:25
	s 12 of 21. From 2018.01.11 14:39:34 to 2018.01.25 08:17:37
	s 13 of 21. From 2018.01.25 08:53:48 to 2018.03.08 23:58:59
	s 14 of 21. From 2018.03.09 00:05:00 

TypeError: test2() got multiple values for argument 'startFrom'

In [4]:
config['assets']=[1,2,3,4,7,8,10,12,13,14,15,16,17,19,27,28,29,30,31,32]

In [7]:
from kaissandra.RNN import modelRNN

data=Data(movingWindow=config['movingWindow'],
              nEventsPerStat=config['nEventsPerStat'],
              lB=config['lB'], 
              dateTest=config['dateTest'],
              assets=config['assets'],
              channels=config['channels'],
              max_var=config['max_var'],
              feature_keys_manual=config['feature_keys_manual'],
              feature_keys_tsfresh=config['feature_keys_tsfresh'],
              var_feat_keys=config['var_feat_keys'])
    
model=modelRNN(data,
                   size_hidden_layer=config['size_hidden_layer'],
                   L=config['L'],
                   size_output_layer=config['size_output_layer'],
                   keep_prob_dropout=config['keep_prob_dropout'],
                   miniBatchSize=config['miniBatchSize'],
                   outputGain=config['outputGain'],
                   commonY=config['commonY'],
                   lR0=config['lR0'],
                   num_epochs=config['num_epochs'])
print(data.feature_keys_manual)
print(len(data.feature_keys_tsfresh)+len(data.var_feat_keys))
print(len(data.var_feat_keys))
print(data.n_feats_tsfresh+len(data.var_feat_keys))
print(data.nFeatures)

[]
54
25
99
99


In [11]:
# test individual assets
assets = [16,17,27,29,30,32]
for asset in assets:
    ass_name = Data().AllAssets[str(asset)]
    try:
        config = configuration({'config_name':'C287INVO_'+ass_name,
                               'assets':[asset],
                               'IDresults':'C287INVO_'+ass_name})
    except OSError:
        config = retrieve_config('C287INVO_'+ass_name)
    test_RNN(config)

Config file ../config/C287INVO_USDCAD.config loaded from disk
IDresults: C287INVO_USDCAD
INFO:tensorflow:Restoring parameters from ../RNN/weights/000287INVO/000007/000007
Parameters loaded. Epoch 7
Epoch 7 of 13. Getting output...
Chunck 1 of 1
Getting results
Epoch = 7. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.5608315, J_train = 4.2515653775989675, Accuracy=0.6803612704391747
RD = 836 NZ = 1561 NZA = 2537 pNZ = 1.336% pNZA = 2.172% AD = 53.56% ADA = 32.95% NO = 329 GSP = 56.84% NSP = 48.33%
SI2 = -32.95 SI = -21.95 eGROI = 1.25% eROI = -2.78% eROI2 = -5.33% eROI3 = -8.62% mSpread = 0.0123%
Epoch = 7. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.6
RD = 298 NZ = 493 NZA = 818 pNZ = 0.422% pNZA = 0.700% AD = 60.45% ADA = 36.43% NO = 205 GSP = 59.02% NSP = 52.20%
SI2 = -12.75 SI = -5.75 eGROI = 3.29% eROI = 0.71% eROI2 = -0.81% eROI3 = -2.86% mSpread = 0.0126%
Epoch = 7. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.7
RD = 107 NZ = 150 NZA = 237 pNZ

C:\Users\mgutierrez\root\Projects\SDC\py\git\results2.py:619: RuntimeWarning: divide by zero encountered in double_scalars
  eROIs = eROIs+GROI-fixed_spread_ratios


Epoch = 7. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.5
RD = 327 NZ = 569 NZA = 837 pNZ = 0.487% pNZA = 0.717% AD = 57.47% ADA = 39.07% NO = 154 GSP = 55.84% NSP = 50.65%
SI2 = -12.70 SI = -6.70 eGROI = 1.58% eROI = -0.17% eROI2 = -1.50% eROI3 = -3.04% mSpread = 0.0113%
Epoch = 7. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.6
RD = 164 NZ = 263 NZA = 391 pNZ = 0.225% pNZA = 0.335% AD = 62.36% ADA = 41.94% NO = 114 GSP = 55.26% NSP = 48.25%
SI2 = -8.70 SI = -7.70 eGROI = 1.28% eROI = -0.07% eROI2 = -1.00% eROI3 = -2.14% mSpread = 0.0119%
Epoch = 7. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.7
RD = 64 NZ = 90 NZA = 126 pNZ = 0.077% pNZA = 0.108% AD = 71.11% ADA = 50.79% NO = 61 GSP = 70.49% NSP = 62.30%
SI2 = 2.45 SI = 4.45 eGROI = 0.87% eROI = 0.18% eROI2 = -0.35% eROI3 = -0.96% mSpread = 0.0114%
Epoch = 7. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.8
RD = 13 NZ = 16 NZA = 19 pNZ = 0.014% pNZA = 0.016% AD = 81.25% ADA = 68.42% NO = 14 GSP = 85.71% N

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch = 7. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.5608315, J_train = 4.2515653775989675, Accuracy=0.680232856775961
RD = 826 NZ = 1516 NZA = 2462 pNZ = 1.298% pNZA = 2.108% AD = 54.49% ADA = 33.55% NO = 321 GSP = 55.14% NSP = 44.86%
SI2 = -44.55 SI = -32.55 eGROI = -0.65% eROI = -4.53% eROI2 = -7.07% eROI3 = -10.28% mSpread = 0.0121%
Epoch = 7. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.6
RD = 267 NZ = 434 NZA = 727 pNZ = 0.372% pNZA = 0.622% AD = 61.52% ADA = 36.73% NO = 183 GSP = 59.56% NSP = 51.37%
SI2 = -11.65 SI = -6.65 eGROI = 3.73% eROI = 1.39% eROI2 = 0.07% eROI3 = -1.76% mSpread = 0.0128%
Epoch = 7. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.7
RD = 113 NZ = 154 NZA = 251 pNZ = 0.132% pNZA = 0.215% AD = 73.38% ADA = 45.02% NO = 88 GSP = 70.45% NSP = 64.77%
SI2 = 6.60 SI = 8.60 eGROI = 2.45% eROI = 1.30% eROI2 = 0.69% eROI3 = -0.19% mSpread = 0.0130%
Epoch = 7. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.8
RD = 23 NZ = 31 N

C:\Users\mgutierrez\root\Projects\SDC\py\git\results2.py:619: RuntimeWarning: invalid value encountered in double_scalars
  eROIs = eROIs+GROI-fixed_spread_ratios


Epoch = 9. Time index = 0. Threshold MC = 0.8. Threshold MD = 0.8
RD = 1 NZ = 1 NZA = 1 pNZ = 0.001% pNZA = 0.001% AD = 100.00% ADA = 100.00% NO = 1 GSP = 100.00% NSP = 100.00%
SI2 = -0.55 SI = 0.45 eGROI = 0.08% eROI = 0.02% eROI2 = 0.06% eROI3 = 0.05% mSpread = 0.0595%
Epoch = 9. Time index = 0. Threshold MC = 0.8. Threshold MD = 0.9
RD = 1 NZ = 1 NZA = 1 pNZ = 0.001% pNZA = 0.001% AD = 100.00% ADA = 100.00% NO = 1 GSP = 100.00% NSP = 100.00%
SI2 = -0.55 SI = 0.45 eGROI = 0.08% eROI = 0.02% eROI2 = 0.06% eROI3 = 0.05% mSpread = 0.0595%

Epoch = 9. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.5
RD = 1 NZ = 1 NZA = 1 pNZ = 0.001% pNZA = 0.001% AD = 100.00% ADA = 100.00% NO = 1 GSP = 100.00% NSP = 100.00%
SI2 = -0.55 SI = 0.45 eGROI = 0.08% eROI = 0.02% eROI2 = 0.06% eROI3 = 0.05% mSpread = 0.0595%
Epoch = 9. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.6
RD = 1 NZ = 1 NZA = 1 pNZ = 0.001% pNZA = 0.001% AD = 100.00% ADA = 100.00% NO = 1 GSP = 100.00% NSP = 100.00%
SI2 = 

KeyboardInterrupt: 